In [1]:
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
#downloads file from given url
def download(url, fpath):
    import requests
    r = requests.get(url)
    with open(fpath, 'wb') as f:
        f.write(r.content)
    return None

#fetching data file
def fetchDatafromWeb(filePath,url):
    download(url,filePath)
    return filePath

In [3]:
url="https://www.cdslab.org/recipes/programming/clustering-kmeans-implementation/points.txt"

In [4]:
fetchDatafromWeb(filePath="./points.txt", url=url)

'./points.txt'

In [5]:
df=pd.read_csv("./points.txt")
df

,x,y
0,3.597482,-0.727244
1,3.171670,-0.602071
2,3.279193,-0.751740
3,3.641694,-0.712578
4,3.283454,-0.872950
...,...,...
1995,-0.891907,1.647074
1996,-0.682693,2.209732
1997,-0.359197,1.894838
1998,-0.691762,1.855902


In [21]:
Pt =np.array (np.transpose([df.x,df.y]))

In [37]:
Pt

array([[ 3.59748245, -0.72724364],
       [ 3.17167032, -0.60207064],
       [ 3.27919288, -0.7517402 ],
       ...,
       [-0.35919741,  1.89483774],
       [-0.69176164,  1.85590248],
       [-0.81944017,  1.54432571]])

In [26]:
#testing getting random points
import random
item = random.choice(tuple(Pt[0:2]))

In [35]:
np.random.seed(1)
s=np.random.choice(len(Pt), size=3, replace=False)
Pt[s,:],s

(array([[ 2.90155748, -3.53972435],
        [-2.36682132,  1.64061962],
        [-1.54713034, -0.88584689]]),
 array([ 674, 1699, 1282]))

In [28]:
type(item)

numpy.ndarray

1. Select the initial medoids randomly (that is, select k points from the dataset randomly as the cluster centers).
2. Iterate while the cost decreases:
 - In each cluster, make the point that minimizes the sum of distances within the cluster the medoid.
 - Reassign each point to the cluster defined by the closest medoid determined in the previous step.

In [63]:
class K_Medoids:
    def __init__(self, k, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        self.medoids_cost= []
        

    def fit_pred(self,data):
        self.initial_medoids= []
        self.centroids = {}
        #randomly initilized centers
        np.random.seed(1)
        samples=np.random.choice(len(data), size=self.k, replace=False)
        self.initial_medoids=data[samples,:]
            
        #for i in range(0,self.k):
            #self.medoids_cost[i].append(0)

        for i in range(self.max_iter):
            self.memberships = {}
        

            for i in range(self.k):
                self.memberships[i] = []
            #compute distances of each point from each cluster
            for i,val in enumerate(data):
                #revised
                #distances = [np.linalg.norm(val-self.centroids[centroid]) for centroid in self.centroids]
                distances = [np.linalg.norm(val-self.initial_medoids[medoid]) for medoid in self.initial_medoids]
                #assigning clusters
                memb = distances.index(min(distances))
                self.memberships[memb].append(val)

            prev_medoids = dict(self.initial_medoids)

            for memb in self.memberships:
                self.initial_medoids[memb] = np.average(self.memberships[memb],axis=0)

            optimized = True

            for m in self.initial_medoids:
                
                original_medoid = prev_medoids[m]
                current_medoid = self.initial_medoids[m]
                if np.sum((current_medoid-original_medoid)/original_medoid*100.0) > self.tol:
                    #print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False
            
            if optimized:
                break

        distances = [np.linalg.norm(data-self.initial_medoids[m]) for m in self.initial_medoids]
        memb = distances.index(min(distances))
        return memb

In [64]:
modl = K_Medoids(k=6)
modl.fit_pred(Pt)

#m= model.fit_pred(data=Pt,k=6)
colormap = np.array(['r', 'g', 'b','y','m','c'])
for m in modl.initial_medoids:
    plt.scatter(modl.initial_medoids[m][0], modl.initial_medoids[m][1],
                marker="o", color="k", s=20, linewidths=5)

for memb in modl.memberships:
    color = colormap[memb]
    for val in modl.memberships[memb]:
        plt.scatter(val[0], val[1], c=color, s=2)
        
plt.show()

IndexError: arrays used as indices must be of integer (or boolean) type